<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/%ED%99%A9%EC%98%81%EC%88%99/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90_AI_Expert_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain 사용환경 설정

*   Langchain과 langchain-openai 라이브러리 패키지 설치
    -  pip install langchain
    -  pip install langchain-openai
* langchain python라이브러리로 프롬프트, 에이전트, 체인 관련 패키지 모음
    - pip install langchainhub

In [ ]:
!pip install langchain
!pip install langchain-openai

In [ ]:
!pip install langchainhub

In [ ]:
from langchain import hub  # 다양한 prompt를 가져오거나 등록해서 사용할 수 있는 패키지

from langchain_openai import ChatOpenAI  # for chat model
from langchain_openai import OpenAI     #for LLM
from langchain_core.pydantic_v1 import BaseModel, Field  # 출력 포맷을 지정을 위해 사용하는 클래스와 메서드 집합
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain_core.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PipelinePromptTemplate,
    FewShotPromptTemplate
)
from langchain_core.output_parsers import (
    StrOutputParser,
    CommaSeparatedListOutputParser,
    JsonOutputParser
)

In [ ]:
# OpenAI 인증키 OPEN_API_KEY 등록
import os

OPENAI_API_KEY = 'Your Key'
organization_id = 'Your Key'


# 기본 LLM



*   ChatOpenAI : 모델을 불러오는 클래스
*   ChatPromptTemplate : prompt 템플릿을 제공해주는 클래스
*   ChatPromptTemplate.from_template() : 문자열 형태의 템플릿을 인자로 받아, 해당 형식에 맞는 프롬프트 객체를 생성
*   StrOutputParaser : 모델을 출력을 문자열 형태로 파싱하여 최종 결과를 반환
*   invoke : chain을 실행하는 메서드
*   batch : Bach Prompt를 구성하고 LLM 호출을 방법


In [ ]:
# ChatOpenAI()를 이용하여 클라이언트 인스턴스를 생성할 때 OPENAI_API_KEY, organization_id, project_id를 입력하여 인스턴스를 생상하는 것으로 해봅니다.

llm = OpenAI(api_key= OPENAI_API_KEY
                , organization=organization_id
                )

In [ ]:
# prompt template 도입
#prompt = ChatPromptTemplate.from_template("You are an expert in Generative AI . Answer the question. <Question>: {input}")

prompt = f"""
  You are an expert in Generative AI . Answer the question.
  <Question> : LangChain을 사용해서 언어모델로 서비스를 만들어보려고 해. LangChain API을 처음 사용하는 사람은 무엇부터 해야하지?
"""

#chain = prompt | llm
llm.invoke(prompt)

- Bach Prompt를 구성하고 LLM 호출을 방법: batch()

In [ ]:
# Batch
prompts = [
    "What is top 5 Korean Street food?",
    "What is most famous place in Seoul?",
    "What is the popular K-Pop group?"
]
llm.batch(prompts)


###  토큰 사용량 추적

   - Langchain 에서는 LLM 제공자가 제공하는 콜백함수를 통해서 LLM 호출건별 토큰 수를 카운트 할 수 있다.
   -  openai에서 제공하는 callback 함수를 이용하여 토큰수를 출력하는 코드:

    - \"with get_openai_callback() as callback:” 코드 블럭내에 LLM 호출 코드를 작성하면 LLM 을 호출한후에, callback에 호출에 대한 메타 정보를 담아서 리턴한다.



In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as callback:
    prompt = "LangChain을 사용해서 언어모델로 서비스를 만들어보려고 해. LangChain API을 처음 사용하는 사람은 무엇부터 해야하지? "
    llm.invoke(prompt)
    print(callback)
    print("Total Tokens:",callback.total_tokens)


In [ ]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125"
                , api_key= OPENAI_API_KEY
                , organization=organization_id
            )

In [ ]:
# Chain으로 연결

# prompt template 도입
prompt = ChatPromptTemplate.from_template("You are an expert in Generative AI . Answer the question. <Question>: {input}")
prompt

chain = prompt | chat_model

chain.invoke({"input": "반도체 제조 산업에서 거대언어 모델을 활용한 대표적인 사례를 알려줘  "})

In [ ]:
# LangChain에서 Prompt를 입력 받아 LLM을 호출하는 방법: 직접 prompt를 전달하거나, ChatModel에서 HumanMessage에 Prompt를 할당하여 호출하는 방식 모두 가능
prompt = "What would be a good company name for a company that makes AI solutions for Semiconductor manufacturing industry?"
messages = [HumanMessage(content=prompt)]

chat_model.invoke(prompt)

# llm.invoke(prompt) 와 llm.invoke(messages) 는 동일한 요청을 LLM에 보낸다.
llm.invoke(messages)

## Prompt Template

*  PromptTemplate은 문자열 프롬프트에 대한 템플릿을 생성하는 데 사용
*  기본적으로 템플릿 작성에는 Python의 str.format 구문을 사용

In [ ]:
# prompt template 사용, template 변수를 동적으로 입력받음
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
text=prompt.format(product="AI solution for manufacturing industry")

llm.invoke(text)

In [ ]:
# prompt template 사용, template 변수를 동적으로 입력받고, 출력 결과를 output parser로 정리하여 결과만 출력

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
text=prompt.format(product="AI solution for manufacturing industry")
output_parser = StrOutputParser()

chain = prompt | llm | output_parser
chain.invoke(text)

In [ ]:
# 문자열 템플릿 결합

template_text = "안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다."

prompt_template = PromptTemplate.from_template(template_text)

filled_prompt = prompt_template.format(name="홍길동", age=30)

print(filled_prompt, "\n", "-"*20)

combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
              + "\n\n{language}로 번역해주세요."
)

combined_text=combined_prompt.format(name="이강산", age=30, language="영어")
print(combined_text, "\n", "-"*20)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
chain.invoke({"age":30, "language":"영어", "name":"홍길동"})




## ChatPromptTemplate

*  각 채팅 메시지는 content와 추가적인 매개변수 role이 연결됨
*  OpenAI Chat Completions API 에서 채팅 메시지는 AI Assitant, Human,  System 역할과 연결

* ChatPromptTemplate는 대화형 상황에서 여러 메시지 입력을 기반으로 단일 메시지 응답을 생성하는 데 사용.

    - ChatPromptTemplate.from_messages : 메시지 리스트(혹은 튜플)을 기반으로 프롬프트를 구성함.
    - ChatPromptTemplate.format_messages : 사용자의 입력을 프롬프트에 동적으로 삽입하여, 최종적으로 대화형 상황을 반영한 메시지 리스트를 생성




```
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
```





In [ ]:
from langchain_core.prompts.chat import ChatPromptTemplate

template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

messages=chat_prompt.format_messages(input_language="English", output_language="Korean", text="I am an AI expert. I love chatting with ChatGPT.")
print(messages)

In [ ]:
chat_model.invoke(messages)

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")
# >> ['hi', 'bye']

In [ ]:
# OpenAI Chat API 프롬프트 세팅
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 월드 클래스 급의 문서 작성 전문가야. 한번 잘 써봐"),
    ("user", "{input}")
])
output_parser = StrOutputParser()

chain = prompt | chat_model | output_parser

# prompt 포함 생성
chain.invoke({"input": "LanhChain과 GPT-4를 이용하여 반도체 제조공정에 대한 교육 자료를 중학생이 이해할 수 있는 수준으로 작성해줘?"})

## LangChain Memory 사용

* 대화 기록을 저장하고, 문맥으로 활용할 수 있음
* langchain.memory 패키지에서 ChatMessageHistory import 하여 사용

  - ChatMessageHistory() : 대화기록을 유지하기 위해 랭체인이 제공하는 클래스
    - add_user_messgge() : 사용자 메시지를 대화기록 객체에 추가하기 위한 메서드
    - add_user_messgge() : ai 가 답변한 대화내용을 대화기록 개첵에 추가하는 메서드


In [ ]:
#ChatMessageHistory is used if you are managing memory outside of a chian directly
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("Where is the top 3 popular space for tourist in Seoul?")
aiMessage = llm.invoke(history.messages)
history.add_ai_message(aiMessage.content)
print(aiMessage.content)
print("-"*20)

history.add_user_message("Which transport can I use to visit the places?")
#print(history.messages)

aiMessage = chat_model.invoke(history.messages)
history.add_ai_message(aiMessage.content)
print(aiMessage.content)


In [ ]:
## 여러분의 대화 시나리오를 만들고 코드로 만들어봅니다.


# LLMChain 과 PromptTemplate 사용

 - LLM Chain은 프롬프트 템플릿을 LLM에 합쳐서 컴포넌트화 한 것으로,
 - 입력값으로 문자열을 넣으면 프롬프트 템플릿에 의해서 프롬프트가 자동으로 완성되고,
 - LLM 모델을 호출하여 텍스트 출력을 내주는 기능을 한다


In [ ]:
#!pip install langchain-community langchain-core

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

#model = ChatOpenAI()
prompt = PromptTemplate.from_template("{city}에서 가장 유명한 관광지 3개를 추천해줘")
chain = LLMChain(llm=llm, prompt=prompt)
city = "서울"
chain.run(city)




# Sequential Chain 사용

* LLMChain 컴포넌트를 만들고 난 뒤, LLMChain들을 서로 연결하는 방법은 여러 Chain을 순차적으로 연결하게 해주는 컴포넌트인 SequentialChain을 사용하는 것이다.

    - 아래 예제는 먼저 도시 이름 {city}을 입력 받은 후에, 첫번째 chain에서 그 도시의 유명한 관광지 이름을 {place}로 출력하도록 한다.
    - 다음 두번째 chain에서는 관광지 이름 {place}를 앞의 chain에서 입력 받고, 추가적으로 교통편 {transport}를 입력받아서, 그 관광지까지 가기 위한 교통편 정보를 최종 출력으로 제공한다.

    - 이때  첫번쨰 chain1 생성시에 output_key를 명시적으로 place로 지정해 준 것에 주의하자.


In [ ]:
from langchain.chains import SequentialChain

prompt1 = PromptTemplate.from_template("{city}에서 가장 유명한 관광명소를 추가적인 설명없이 장소 이름만으로 추천해줘.")
prompt2 = PromptTemplate.from_template("{place}을 {transport} 편으로 가는 방법을 알려줄래?")
chain1 = LLMChain(llm=llm,prompt=prompt1,output_key="place",verbose=True)
chain2 = LLMChain(llm=llm,prompt=prompt2,verbose=True)

chain = SequentialChain(chains=[chain1,chain2], input_variables=["city","transport"],verbose=True)
chain.run({'city':'서울','transport':'전철'})


# Prompt Serialization

 프롬프트를 수시로 변경해야 하는 경우에 프롬프트 템플릿을 별도의 파일로 저장하여, 애플리케이션에서 로드하여 사용하는 방법

* 프롬프트 템플릿을 저장하는 방법
 - PrompteTemplate객체를 생성한후에 save(“{JSON 파일명}”)을 해주면 해당 템플릿이 파일로 저장

* 저장된 프롬프트 템플릿을 Load하여 사용하기
  - 저장된 템플릿은 langchain.prompts 패키지의 load_prompt 함수를 이용하여 다시 불러와 사용 가능


In [ ]:
from langchain import PromptTemplate

template = PromptTemplate.from_template(
    "Tell me a {adjective} {topic} in {city} in 300 characters."
)
template.save("template.json")



In [ ]:
from langchain.prompts import load_prompt

loaded_template = load_prompt("template.json")
prompt = loaded_template.format(adjective="popular", topic="cafe", city="San francisco")
print(prompt)

output_parser = StrOutputParser()

chain = llm | output_parser
chain.invoke(prompt)

* FewShotPromptTemplate

In [ ]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 스티브 잡스는 몇 살에 사망했나요?
                  중간 답변: 스티브 잡스는 56세에 사망했습니다.
                  추가 질문: 아인슈타인은 몇 살에 사망했나요?
                  중간 답변: 아인슈타인은 76세에 사망했습니다.
                  최종 답변은: 아인슈타인
                  """,
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 네이버의 창립자는 누구인가요?
                  중간 답변: 네이버는 이해진에 의해 창립되었습니다.
                  추가 질문: 이해진은 언제 태어났나요?
                  중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
                  최종 답변은: 1967년 6월 22일
                  """,
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 율곡 이이의 어머니는 누구인가요?
                  중간 답변: 율곡 이이의 어머니는 신사임당입니다.
                  추가 질문: 신사임당은 언제 태어났나요?
                  중간 답변: 신사임당은 1504년에 태어났습니다.
                  추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
                  중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
                  최종 답변은: 연산군
                  """,
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 올드보이의 감독은 누구인가요?
                  중간 답변: 올드보이의 감독은 박찬욱입니다.
                  추가 질문: 박찬욱은 어느 나라 출신인가요?
                  중간 답변: 박찬욱은 대한민국 출신입니다.
                  추가 질문: 기생충의 감독은 누구인가요?
                  중간 답변: 기생충의 감독은 봉준호입니다.
                  추가 질문: 봉준호는 어느 나라 출신인가요?
                  중간 답변: 봉준호는 대한민국 출신입니다.
                  최종 답변은: 예
                  """,
    },
]

* PromptTemplate의 input_variables, template 을 이용하여 prompt를 example_prompt를 생성하고

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {question}",
    input_variables=["question"], #프롬프트가 수신할 입력 항목
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

In [ ]:
chain = llm | StrOutputParser()

print(chain.invoke(final_prompt))

# 모델 파라미터 주기

In [ ]:
# 모델 생성 단계에서 주기
params = {
    "temperature": 0.7,
    "max_tokens": 100,
}

kwargs = {
    "frequency_penalty": 0.5,
    "presence_penalty": 0.5,
    "stop": ["\n"]

}

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", **params, model_kwargs = kwargs)

question = "태양계에서 가장 큰 행성은 무엇인가요?"
response = llm.invoke(input=question)

print(response)

In [ ]:
# 모델 호출 단계에서 주기
params = {
    "temperature": 0.7,
    "max_tokens": 10,
}

response = llm.invoke(input=question, **params)

print(response.content)

Bind Method를 이용한 파라미터 추가 설정

In [ ]:
# bind 메서드를 통해 모델 인스턴스에 파라미터를 추가로 제공할 수 있음
# 특수한 상황에서만 일부 파라미터를 다르게 적용할 수 있도록 해줌.

prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=100)

messages = prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")

before_answer = model.invoke(messages)

print(before_answer)

chain = prompt | model.bind(max_tokens=10)

after_answer = chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

print(after_answer)

# Parser

*   CSV Parser
  *  CommaSeparatedListOutputParser : 모델이 생성한 텍스트에서 쉼표로 구분된 항목을 추출
  * get_format_instructions : 모델에 전달할 포멧 지시사항을 얻는 메서드
*   Json Parser
  * JsonOutputParser : 모델의 출력을 json으로 해석하며, Pydantic 모델에 맞게 데이터를 구조화하여 제공
  * get_format_instructions : 모델에 전달할 포멧 지시사항을 얻는 메서드


# 프롬프트 조합과 Partial Prompt Template 사용

* Partial Prompt Template
  - 프롬프트의 변수 값을 한번에 채워 넣는 것이 아니라, 이 중 일부만 먼저 채워 넣고 나머지는 나중에 채워 넣는 방식

  - 애플리케이션 코드내에서 템플릿 생성시 이미 알고 있는 값이 있을때 사용하기 편리한 방식





- 콤마로 분리된 형태로 결과를 출력하는 CommaSeparatedListOutputParser

In [ ]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

In [ ]:
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

chain = prompt | llm | output_parser

chain.invoke({"subject": "popular Korean cusine"})


 - JSON 포맷을 사용하여 출력하는 JsonOutpoutParser
    - PydanticOutputParser를 이용하여 출력을 변환하여 원하는 구조 정보로 정의하여 사용
    - 원하는 구조정보 정의는 BaseModel과 Field를 이용하여 클래스로 정의하여 전달함

In [ ]:
#자료구조 정의
class CusineRecipe(BaseModel):
    name: str = Field(description="name of a cusine")
    recipe: str = Field(description="recipe to cook the cusine")

output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()  # get_formet_instuctions() : 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공

print(format_instructions)

# LangChain Agents

In [ ]:
!pip install wikipedia

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0,  # 창의성 0으로 설정
                 model_name='gpt-4',  # 모델명
                )

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

tools = load_tools(["wikipedia", "llm-math"], llm=llm) #llm-math의 경우 나이 계산을 위해 사용
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         description='위키피이아에서 정보를 검색하고 계산이 필요할 때 사용',
                         verbose=True)


agent.run("gpt-4o는 언제 출시되었어?")

#agent.run("에드 시런이 태어난 해는? 2024년도 현재 에드 시런은 몇 살?")